In [1]:
import attr
import numpy as np
import warnings

from gibbs.models.ceos import PengRobinson78
from gibbs.mixture import Mixture
from gibbs.stability_analysis import stability_test

warnings.filterwarnings('ignore')

* Case 1:

In [2]:
@attr.s(auto_attribs=True)
class NichitaModel1:
    z: np.ndarray
    Tc: np.ndarray
    Pc: np.ndarray
    acentric_factor: np.ndarray
    bip: np.ndarray
        
    @property
    def mixture(self):
        return Mixture(
            z=self.z, 
            Tc=self.Tc, 
            Pc=self.Pc, 
            acentric_factor=self.acentric_factor
        )

    @property
    def model(self):
        return PengRobinson78(
            mixture=self.mixture,
            bip=self.bip
        )

    @property
    def number_of_components(self):
        return len(self.z)

    def fugacity(self, P, T, z):
        Z_factor = self.calculate_Z(P, T, z)
        return self.model.calculate_fugacity(P, T, z, Z_factor)

    def calculate_Z(self, P, T, z):
        Z_factor = self.model.calculate_Z_minimal_energy(P, T, z)
        return Z_factor

In [ ]:
z = np.array([0.10, 0.60, 0.30])
omegas = np.array([0.0115, 0.1928, 0.4902])
Tcs = np.array([190.556, 425.16667, 617.666667])
Pcs = np.array([4604318.9, 3796942.8, 2.096e6])
kijs = np.zeros((3, 3))
model = NichitaModel1(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)